# JAS 1101 Group Project - Lightcurve Generation Notebook

### Team 3: Shannon Bowes, Westley Brown, Marco Gallegos Herrada, Mattias Lazda

In [1]:
import phoebe as pb
from phoebe import u
import numpy as np
import matplotlib.pyplot as plt

This next cell has to be run in order to get around a bug that is in the PHOEBE code itself (see the latest Git issue tag for explanation).

In [2]:
pb.atmospheres.passbands._url_tables_server = 'https://staging.phoebe-project.org'
pb.list_online_passbands(refresh=True)
pb.list_all_update_passbands_available()
pb.update_all_passbands()

This initiates a default binary system that you can change the parameter values of. The dataset is an empty dataset that generates a list of times and any changes to parameters will be added to this dataset. If we wanted to plot our synthetic lightcurve over an observed lightcurve, this is how we would add the observed data.

We can change our sampling by adjusting the number of times in `times=pb.linspace(0,2.71,x)`.

In [17]:
logger = pb.logger()
b = pb.default_binary()
b.add_dataset('lc', times=pb.linspace(0,2.71,1000), dataset='lc01', overwrite=True)

<ParameterSet: 80 parameters | contexts: constraint, figure, compute, dataset>

There's a couple different ways that you can set parameter values. Below is how I set them.
There's a lot of parameters that are set to default values, so you don't need to set them all.

If you want to see what the default values are, you can use the command `print(b.filter())` to see all the parameters that are set to default values. You can also do this after setting parameters to see the final values of all the parameters in your 'bundle' (the PHOEBE term for model).
Changing the value of a parameter is as simple as `b['parameter_name'] = value`. For example, `b['q'] = 0.5` would set the mass ratio to 0.5.
However, you often have to specify whether the parameter you are referencing is for the primary or secondary components or the binary overall (so of the form `b['q@binary']`).

You cannot set some parameters immediately without addressing hierarchy. For example, you cannot set `b['teffratio@binary'] = 0.65` because `teffratio` is a constrained parameter of the primary and secondary components, defined by the physical equations written into PHOEBE. You would need to flip the equation, by running `b.flip_constraint("teffratio@binary", solve_for = "teff@primary")`, for example, instead. We'll know we're running into this problem if we get the error "ValueError: cannot change the value of a constrained parameter." If we run into issues, we can look at this tutorial:
https://phoebe-project.org/docs/2.4/tutorials/constraints_hierarchies

In [18]:
#setting chosen binary parameters
b["ecc@binary"] = 1.6E-4
b["q@binary"] = 1
b["incl@binary"] = 77.3
b["period@binary"] = 2.71

### when you flip it, if you rerun the cell you have to comment out the lines because flipping it back is a different command and we dont want to do that
# print(b["requivsumfrac@binary"])
# print(b["requivsumfrac@binary@orbit@component"].constrained_by)

### comment out these two lines after running once (if re-running cell)
b.flip_constraint("requivsumfrac@binary", solve_for="requiv@primary")
b.flip_constraint("teffratio@binary", solve_for = "teff@primary")

b['teffratio@binary'] = 0.65
b["requivsumfrac@binary@orbit@component"] = 0.483

b["gravb_bol@primary"] = 0.9
b["irrad_frac_refl_bol@primary"] = 0.9
print(b.run_checks())

print(b.filter())

Run Checks Report: PASS

ParameterSet: 187 parameters
                        t0@system: 0.0 d
                        ra@system: 0.0 deg
                       dec@system: 0.0 deg
                  distance@system: 1.0 m
                    vgamma@system: 0.0 km / s
C                      ebv@system: 0.0
                        Av@system: 0.0
                        Rv@system: 3.1
                 hierarchy@system: orbit:binary(star:primary, star:secondary)
C        requiv@primary@component: 1.5598999999999998 solRad
C    requiv_max@primary@component: 2.0130111426796167 solRad
C          teff@primary@component: 9230.76923076923 K
           abun@primary@component: 0.0
C          logg@primary@component: 3.1854197703611282
        syncpar@primary@component: 1.0
C        period@primary@component: 2.71 d
C          freq@primary@component: 2.318518563534903 rad / d
          pitch@primary@component: 0.0 deg
            yaw@primary@component: 0.0 deg
C          incl@primary@component: 77.3 

At this point, as an extra sanity check, we should check that the parameters are indeed what we set them to be (when setting multiple parameters at once, if you are not paying attention to hierarchies, you may accidentally reset a parameter you set previously).

In [5]:
print(b["ecc@binary"])
print(b["q@binary"])
print(b["incl@binary"])
print(b["period@binary"])
print(b['teffratio@binary'])
print(b["requivsumfrac@binary@orbit@component"])

Parameter: ecc@binary@component
                       Qualifier: ecc
                     Description: Eccentricity
                           Value: 0.00016
                  Constrained by: 
                      Constrains: t0_perpass@binary@component, t0_ref@binary@component, ecosw@binary@component, esinw@binary@component, requiv_max@primary@component, requiv_max@secondary@component
                      Related to: t0_supconj@binary@component, period@binary@component, per0@binary@component, dpdt@binary@component, dperdt@binary@component, t0@system, t0_perpass@binary@component, t0_ref@binary@component, ecosw@binary@component, esinw@binary@component, q@binary@component, syncpar@primary@component, sma@binary@component, incl@primary@component, long_an@primary@component, incl@binary@component, long_an@binary@component, requiv_max@primary@component, syncpar@secondary@component, incl@secondary@component, long_an@secondary@component, requiv_max@secondary@component

Parameter: q@binary@co

This next cell actually computes the model:

In [19]:
# b.run_compute(irrad_method='none')
b.run_compute()
b.plot(x='phase', save = 'pbtest2.png')
plt.show()

100%|██████████| 1000/1000 [00:21<00:00, 46.48it/s]
/var/folders/34/3847lqd14j78mfm70_z1c_r00000gn/T/ipykernel_50904/2596823045.py:4: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()


This next cell allows you to view the lightcurve in the figure 'pbtest.png' which will be in the same directory as where you run this file. There's some matplotlib issue that I can't figure out how to solve right now that doesn't allow the figure to be shown in this notebook.

In [7]:
plotter = b.plot(x='phase',  ylabel = 'Relative flux', xlabel = 'Phase', show=True, save = 'pbtest.png')
# plt.ion()
plt.show(block=False)

/opt/anaconda3/envs/phoebe_environment/lib/python3.13/site-packages/phoebe/dependencies/autofig/figure.py:627: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show()  # <-- blocking
/var/folders/34/3847lqd14j78mfm70_z1c_r00000gn/T/ipykernel_50904/1770872199.py:3: UserWarning: FigureCanvasAgg is non-interactive, and thus cannot be shown
  plt.show(block=False)


You can then save an output file (format is by default JSON) of the binary model:

In [11]:
print(b.save('model.JSON'))

model.JSON


In [20]:
# extract fluxes as a fxn of time then cubic spline interpolate 

Statistical methods can now be performed on this file, such as MCMC (see https://phoebe-project.org/docs/2.4/tutorials/emcee).

### Below is my trial zone of different things - may not work and may not be included in our final outcomes.

In [12]:
# b.add_dataset('mesh', times=pb.linspace(0,2.71,101), columns=['teffs'])
b.add_dataset('mesh', times=[0, 0.50, 1.0, 1.5, 2.0, 2.5], columns=['teffs'])

Thu, 20 Feb 2025 11:46 BUNDLE       WARNING mesh dataset uses 'compute_times' instead of 'times', applying value sent as 'times' to 'compute_times'.


<ParameterSet: 10 parameters | contexts: constraint, compute, dataset>

In [13]:
afig, mplfig = b.filter(time=1.0).plot(fc='teffs', save='trial.png')

ValueError: Nothing could be found to plot.  Check all arguments.